In [ ]:
!pip install boto3
!pip install hana_ml
!pip install replace

In [ ]:
#For DWC
import boto3
import replace
import pandas as pd
import sys
import os
import hana_ml.dataframe as dataframe
#Establish DWC Space Open SQL Connection
conn = dataframe.ConnectionContext(address = '', #Provide your OpenSQL Host here
                                   port = 443, 
                                   user = '', #Provide your OpenSQL container Username here
                                   password = '', #Provide your OpenSQL contianer password here
                                   encrypt = 'true'
                                  )

#Get the Access Keys and Secret Keys for AWS S3 Buckets
session = boto3.Session( 
         aws_access_key_id='', #Provide your AWS Access key here
         aws_secret_access_key='')#Provide your AWS Secret  key here

#Then use the session to get the resource
s3 = session.resource('s3')
#provide the S3 bucketname
my_bucket = s3.Bucket('dwcdmp') #Provide your AWS S3 Bucketname here
#loop around all files in the bucket and insert it into DWC Space Open SQL container
#Make sure you just have csv files in the bucket mentioned at line 24
for objects in my_bucket.objects.all():
    key = objects.key
#replace .csv from the file name while ingesting the csvs as tables in DWC 
    keystr = key.replace(".csv","")
    print(keystr)
    csvfile = pd.read_csv(objects.get()['Body'])
    print (csvfile)
    df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = csvfile, 
                                                   table_name = keystr,
                                                   force = True,
                                                   replace = True)